In [1]:
import numpy as np
import pandas as pd
from scipy import stats as st
import itertools
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from joblib import dump, load

In [8]:
def sliding_windows(df, features, window_size=20):
    """Create a sliding window with a defined window size and return the calculation for each record inside the sliding window.

    The calculations made are for each time window: 
    - The mean.
    - The median.
    - The standard deviation.
    - The maximum and minimum value.
    - The trend.

    Note the process is explained in greater detail in: <TODO: reference link>

    Args:
        dataset (numpy.array): An array with the data taken on a vehicle trip, 
        composed for rows: dataset registers and columns: first data index, and subsequent dataset event features.
        window_size (int, optional): number Number of registers contained in the time window. Defaults to 10.
        event_features (str, optional): the names of dataset features. Defaults to "X".

    Returns:
        tuple: A tuple structured like this: (sliding window id, sliding window featured data, sliding window label data, features name)
    """
    features = ["id", *features,  "eventClass"]
    dataset = df[features].to_numpy()

    sld_window = np.lib.stride_tricks.sliding_window_view(dataset, window_size, axis=0) #[::1, :] Add this for define window step

    # Splitin dataset id
    id = sld_window[:,0:1,:]
    sld_window_id = np.concatenate((id[:,:,0], id[:,:,-1]), axis=1) # get the first and last id from registers in every sliding window

    # Spliting the dataset (features, label)
    separator = dataset.shape[1] - 1 # Split the last page corresponding to the eventClass
    features_data = sld_window[:, 1:separator, :] # Get the features of the data in every sliding window
    label_data = sld_window[:, separator, :] # Get the labels of the data in every sliding window

    # Processing the sliding window
    # Get the mean, median, std, max and min value
    mean = features_data.mean(axis=2)
    median = np.median(features_data, axis=2)
    std = features_data.std(axis=2)
    max_val = features_data.max(axis=2)
    min_val = features_data.min(axis=2)
    # Get tendency
    divider = np.array([mean[0], *mean[:-1]])
    tendency = mean/np.where(divider == 0, 1, divider)
    label = st.mode(label_data, axis=1)[0]

    # Concatenate processed sliding window
    sld_window_features = np.concatenate((mean, median, std, max_val, min_val, tendency), axis=1)
    # Reshape label for sklearn standard
    label = label.reshape(label.shape[0])

    # Make input algorithm df
    X = pd.DataFrame(sld_window_features)
    X[["first","last"]] = sld_window_id
    X.set_index(["first","last"], inplace=True)

    return (X, label)

In [3]:
from pykalman import KalmanFilter

def data_filter(df):
    features = ["accX", "accY", "velAngZ", "magX", "magY"]

    for var in features:
        data = df[var]
        
        # Kalman filter process
        kf = KalmanFilter(initial_state_mean = data.iloc[0], n_dim_obs=1)
        filter_data = kf.em(data).filter(data)[0].T[0]
        filter_data_s = pd.Series(np.array(filter_data), name=var)
        df[var] = filter_data_s

        # Normalized magnetometer data with min-max normalization
        if var == "magX" or var =="magY":
            normalized_data = (data-data.min())/(data.max()-data.min())
            df[var] = normalized_data

    return df

## Prueba

In [5]:
clasifiers = ["clf_sudden_braking_smartphone",
              "clf_sudden_braking_raspberry",
              "clf_sudden_acceleration_smartphone",
              "clf_sudden_acceleration_raspberry",
              "clf_chg_line_right_smartphone",
              "clf_chg_line_right_raspberry",
              "clf_chg_line_left_smartphone",
              "clf_chg_line_left_raspberry",
              "clf_agg_turn_right_smartphone",
              "clf_agg_turn_right_raspberry",
              "clf_agg_turn_left_smartphone",
              "clf_agg_turn_left_raspberry"]

data = ["smartphone_02-Feb-2022-20-01_frenada repentina_Data-Mv2EUSpYPrGpMG3qjaD","raspberry_02-Feb-2022-20-02_frenada repentina_Data-Mv2E_1BnfkXOP6gRM1J",
        "smartphone_02-Feb-2022-19-59_aceleración repentina_Data-Mv1jxE4ZYJvJ9AVKae1", "raspberry_02-Feb-2022-20-00_aceleración repentina_Data-Mv1m25ILfD2v20bDi5Q",
        "smartphone_02-Feb-2022-20-26_cambio de línea agresivo derecha_Data-Mv2MrzIVJOQsCqDJHcb", "raspberry_02-Feb-2022-20-29_cambio de línea agresivo derecha_Data-Mv2N4sqCOBh_ItfGkmE",
        "smartphone_02-Feb-2022-20-21_cambio de línea agresivo izquierda_Data-Mv2LixfwEokL4QCgN_p", "raspberry_02-Feb-2022-20-22_cambio de línea agresivo izquierda_Data-Mv2LjjP0SLKY6zw_7Y9",
        "smartphone_02-Feb-2022-20-31_giro derecho agresivo_Data-Mv2Nk_4aV4M_KufahA0", "raspberry_02-Feb-2022-20-08_giro derecho agresivo_Data-Mv2HSOB7ABwlXlcqhJ8",
        "smartphone_02-Feb-2022-20-15_giro izquierdo agresivo_Data-Mv2KZyC2nSbvYeWyEmh", "raspberry_02-Feb-2022-20-15_giro izquierdo agresivo_Data-Mv2JoflmGmLbEwyIi2a"]

features = [["speed","accY"],
            ["speed","accY"],
            ["speed","accY"],
            ["speed", "accPosition", "accY"],
            ["accX", "velAngZ"],
            ["accX", "velAngZ"],
            ["accX", "velAngZ"],
            ["accX", "velAngZ"],
            ["accX" ,"accY", "velAngZ", "magX", "magY"],
            ["speed", "accX", "accY", "velAngZ", "magX"],
            ["accX" ,"accY", "velAngZ", "magX", "magY"], 
            ["speed", "accX", "accY", "velAngZ", "magX"]]

for c, d, f in zip(clasifiers, data, features):
    print("\n\n", c,"\n\n")
    clf = load(str("./built_algorithms/"+c+".joblib"))
    df = pd.read_csv(str("./filtered_data/"+d+".csv"))
    event_features_label = ["id", *f,  "eventClass"]
    dataset = df[event_features_label].to_numpy()
    window_size = 40
    ids, X, y, features_names = sliding_windows(dataset, window_size, f)
    X_df = pd.DataFrame(X, columns=features_names)
    X_df[["first","last"]] = ids
    X_df.set_index(["first","last"], inplace=True)

    y_predict = clf.predict(X_df.values)
    y_predict_proba = clf.predict_proba(X_df.values)[:,1]
    
    print('Accuracy: {:.6f}'.format(accuracy_score(y, y_predict)))
    print('Precision: {:.6f}'.format(precision_score(y, y_predict)))
    print('Recall: {:.6f}'.format(recall_score(y, y_predict)))
    print('F1: {:.6f}'.format(f1_score(y, y_predict)))
    print('AUC: {:.6f}'.format(roc_auc_score(y, y_predict_proba)))



 clf_sudden_braking_smartphone 



Sliding windows shape:  (414, 4, 40)
Accuracy: 1.000000
Precision: 1.000000
Recall: 1.000000
F1: 1.000000
AUC: 1.000000


 clf_sudden_braking_raspberry 



Sliding windows shape:  (401, 4, 40)
Accuracy: 1.000000
Precision: 1.000000
Recall: 1.000000
F1: 1.000000
AUC: 1.000000


 clf_sudden_acceleration_smartphone 



Sliding windows shape:  (382, 4, 40)
Accuracy: 1.000000
Precision: 1.000000
Recall: 1.000000
F1: 1.000000
AUC: 1.000000


 clf_sudden_acceleration_raspberry 



Sliding windows shape:  (296, 5, 40)
Accuracy: 1.000000
Precision: 1.000000
Recall: 1.000000
F1: 1.000000
AUC: 1.000000


 clf_chg_line_right_smartphone 



Sliding windows shape:  (624, 4, 40)
Accuracy: 1.000000
Precision: 1.000000
Recall: 1.000000
F1: 1.000000
AUC: 1.000000


 clf_chg_line_right_raspberry 



Sliding windows shape:  (388, 4, 40)
Accuracy: 1.000000
Precision: 1.000000
Recall: 1.000000
F1: 1.000000
AUC: 1.000000


 clf_chg_line_left_smartphone 



Sliding windows

## Algoritmo de reconocimiento de near-crash

In [16]:
# TODO: this variables is getting by firebase in app.py
device_name = "smartphone" #"raspberry"
firebase_data = "./data/smartphone_02-Feb-2022-20-01_frenada repentina_Data-Mv2EUSpYPrGpMG3qjaD.csv" # TODO: change for data getting by firebase
df = pd.read_csv(firebase_data)


clasifiers = ["clf_sudden_braking_smartphone",
              "clf_sudden_braking_raspberry",
              "clf_sudden_acceleration_smartphone",
              "clf_sudden_acceleration_raspberry",
              "clf_chg_line_right_smartphone",
              "clf_chg_line_right_raspberry",
              "clf_chg_line_left_smartphone",
              "clf_chg_line_left_raspberry",
              "clf_agg_turn_right_smartphone",
              "clf_agg_turn_right_raspberry",
              "clf_agg_turn_left_smartphone",
              "clf_agg_turn_left_raspberry"]

features = [["speed","accY"],
            ["speed","accY"],
            ["speed","accY"],
            ["speed", "accPosition", "accY"],
            ["accX", "velAngZ"],
            ["accX", "velAngZ"],
            ["accX", "velAngZ"],
            ["accX", "velAngZ"],
            ["accX" ,"accY", "velAngZ", "magX", "magY"],
            ["speed", "accX", "accY", "velAngZ", "magX"],
            ["accX" ,"accY", "velAngZ", "magX", "magY"], 
            ["speed", "accX", "accY", "velAngZ", "magX"]]

if device_name == "smartphone":
    clasifiers = clasifiers[::2]
    features = features[::2]
else:
    clasifiers = clasifiers[1::2]
    features = features[1::2]

# TODO: before filter is need to manage the offset of the data, for this reason in the experiments we need to make a standby time
"""max_standby = 7660
var_with_offset = "accY"
offset = df.loc[df['id'] <= max_standby, var_with_offset].mean()
df[var_with_offset] = df[var_with_offset] - offset"""


# Make filter kalman for all data
df_filtered = data_filter(df)

near_crash = {}

for c, f in zip(clasifiers, features):
    #Make sliding window
    X, y = sliding_windows(df_filtered, f, 40)

    # Check near-crash
    clf = load(str("./built_algorithms/"+c+".joblib"))
    y_predict = clf.predict(X.values)
    y_predict_proba = clf.predict_proba(X.values)[:,1]

    if (len(np.where(y_predict == 1.0)[0]) != 0):
        near_crash_df = X.iloc[np.where(y_predict == 1.0)[0]]
        near_crash[c] = near_crash_df

near_crash

{'clf_sudden_braking_smartphone':                          0         1          2         3         4         5  \
 first   last                                                                    
 33026.0 33065.0  28.034598 -2.693644  26.896181 -0.101513  1.227545  3.286733   
 33027.0 33066.0  28.012121 -2.829017  26.896181 -0.287917  1.200094  3.273355   
 33028.0 33067.0  27.989644 -2.942426  26.896181 -2.395554  1.171569  3.245156   
 33029.0 33068.0  27.967167 -3.028473  26.896181 -3.768366  1.141889  3.204784   
 33030.0 33069.0  27.944690 -3.084421  26.896181 -3.768366  1.110963  3.158017   
 33031.0 33070.0  27.922213 -3.109734  26.896181 -3.768366  1.078682  3.135164   
 33032.0 33071.0  27.976214 -3.114662  27.976214 -3.768366  1.080033  3.130625   
 33033.0 33072.0  27.838813 -3.126082  27.976214 -3.768366  1.483088  3.119935   
 33034.0 33073.0  27.701412 -3.138076  27.976214 -3.768366  1.787419  3.108179   
 33035.0 33074.0  27.564012 -3.155055  27.976214 -3.768366  2.037

In [10]:
from itertools import groupby
from operator import itemgetter
data = [1,2,3,4,5,6,7,8,10,11,12,13,14,15,17,30,31,32,60,61,62,63,64,65,70,71,80,81,82,83,84,85]
for k, g in groupby(enumerate(data), lambda i_x: i_x[0] - i_x[1]):
    print(list(map(itemgetter(1), g)))

[1, 2, 3, 4, 5, 6, 7, 8]
[10, 11, 12, 13, 14, 15]
[17]
[30, 31, 32]
[60, 61, 62, 63, 64, 65]
[70, 71]
[80, 81, 82, 83, 84, 85]


In [17]:
import numpy as np
data = np.sort(np.unique(np.random.randint(40, size=40)))
data2 = np.sort(np.unique(np.random.randint(20, size=20)))
def consecutive(data, stepsize=1):
    return np.split(data, np.where(np.diff(data) != stepsize)[0]+1)

a = np.array(data)
b = np.array(data2)
nc = consecutive(a)
nc2 = consecutive(b)
print(nc)
"""
i = 0
while (len(nc) != 0):
    if (nc[i][-1] + 2.0) >= nc[i+1][0]:
        nc[i] = (np.concatenate((nc[i], nc[i+1])))
        del nc[i+1]
        i += 1
    else:
        nc[i] = nc[i]
        i += 1
    print("\n\n")
    print(nc)

print(nc)"""
print("\n\nnew")
new = [nc[0]]
newY = [nc2[0]]

for i, d in enumerate(nc[:-1]):
    if (nc[i][-1] + 2) >= nc[i+1][0]:
        new[-1] = np.concatenate((new[-1], nc[i+1]))
    else:
        new.append(nc[i+1])
print(new)

for i, d in enumerate(nc2[:-1]):
    if (nc2[i][-1] + 2) >= nc2[i+1][0]:
        newY[-1] = np.concatenate((newY[-1], nc2[i+1]))
    else:
        newY.append(nc2[i+1])
print("\n\nnewY")
print(newY)

new2 = []
newY2 = []
for d in new:
    start = d[0]
    end = d[-1]

    if (end + 2) - start >= 3:
        new2.append(d)

for d in newY:
    start = d[0]
    end = d[-1]

    if (end + 2) - start >= 3:
        newY2.append(d)

print("\nnew2")
print(new2)
print("\nnewY2")
print(newY2)




[array([4, 5, 6, 7, 8]), array([10]), array([12]), array([14, 15, 16, 17]), array([19, 20, 21, 22, 23, 24]), array([26, 27, 28, 29]), array([32]), array([34]), array([36, 37, 38, 39])]


new
[array([ 4,  5,  6,  7,  8, 10, 12, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24,
       26, 27, 28, 29]), array([32, 34, 36, 37, 38, 39])]


newY
[array([ 1,  2,  4,  5,  6,  7,  9, 10, 11, 12, 14]), array([18, 19])]

new2
[array([ 4,  5,  6,  7,  8, 10, 12, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24,
       26, 27, 28, 29]), array([32, 34, 36, 37, 38, 39])]

newY2
[array([ 1,  2,  4,  5,  6,  7,  9, 10, 11, 12, 14]), array([18, 19])]


In [19]:
np.unique(np.concatenate((new2[0], newY2[0])))

array([ 1,  2,  4,  5,  6,  7,  8,  9, 10, 11, 12, 14, 15, 16, 17, 19, 20,
       21, 22, 23, 24, 26, 27, 28, 29])

In [ ]:
"Near crash select before check size of data (tamaño [25, 111, 16, 27, 27, 16, 15, 74])"
[array([15172., 15173., 15174., 15175., 15176., 15177., 15178., 15179.,
       15180., 15181., 15182., 15183., 15184., 15185., 15186., 15187.,
       15188., 15189., 15190., 15191., 15192., 15193., 15203., 15204.,
       15205.]),
       
array([15346., 15347., 15348., 15349., 15350., 15351., 15352., 15353.,
       15354., 15355., 15356., 15357., 15358., 15359., 15360., 15361.,
       15362., 15363., 15364., 15365., 15366., 15367., 15368., 15369.,
       15370., 15371., 15372., 15373., 15374., 15375., 15376., 15383.,
       15384., 15385., 15386., 15387., 15389., 15390., 15391., 15421.,
       15422., 15423., 15424., 15425., 15426., 15427., 15428., 15429.,
       15430., 15431., 15432., 15433., 15434., 15435., 15436., 15437.,
       15438., 15439., 15440., 15441., 15442., 15443., 15444., 15445.,
       15446., 15447., 15448., 15449., 15450., 15451., 15452., 15453.,
       15454., 15455., 15456., 15457., 15458., 15459., 15460., 15461.,
       15462., 15463., 15464., 15465., 15466., 15467., 15468., 15469.,
       15470., 15471., 15472., 15473., 15474., 15475., 15476., 15477.,
       15478., 15479., 15480., 15481., 15482., 15483., 15484., 15485.,
       15486., 15487., 15488., 15489., 15490., 15491., 15492.]), 
array([15762., 15763., 15764., 15768., 15769., 15770., 15771., 15772.,
       15773., 15774., 15775., 15776., 15777., 15778., 15779., 15780.]),
       
array([15842., 15843., 15844., 15845., 15846., 15847., 15848., 15849.,
       15853., 15854., 15855., 15856., 15857., 15858., 15859., 15860.,
       15861., 15862., 15863., 15864., 15865., 15866., 15873., 15874.,
       15875., 15876., 15877.]),
       
array([16342., 16343., 16344., 16345., 16346., 16347., 16348., 16349.,
       16350., 16351., 16352., 16353., 16354., 16355., 16356., 16357.,
       16358., 16359., 16360., 16361., 16362., 16363., 16364., 16365.,
       16366., 16367., 16368.]),
       
array([16534., 16535., 16536., 16538., 16539., 16540., 16541., 16542.,
       16543., 16544., 16545., 16546., 16547., 16548., 16549., 16550.]),
       
array([17211., 17212., 17213., 17214., 17215., 17216., 17217., 17218.,
       17219., 17220., 17221., 17222., 17223., 17224., 17225.]),
       
array([18019., 18020., 18021., 18022., 18023., 18024., 18025., 18026.,
       18027., 18028., 18029., 18030., 18031., 18032., 18033., 18034.,
       18035., 18036., 18037., 18038., 18039., 18040., 18041., 18042.,
       18043., 18044., 18045., 18046., 18047., 18048., 18049., 18050.,
       18051., 18052., 18053., 18054., 18055., 18056., 18057., 18058.,
       18059., 18060., 18061., 18062., 18063., 18064., 18065., 18104.,
       18105., 18106., 18107., 18108., 18109., 18110., 18111., 18112.,
       18113., 18114., 18115., 18116., 18117., 18118., 18119., 18120.,
       18121., 18122., 18123., 18124., 18125., 18126., 18127., 18128.,
       18132., 18133.])]

In [ ]:
"[dict_keys(['clf_sudden_braking_raspberry', 'clf_sudden_acceleration_raspberry', 'clf_chg_line_right_raspberry', 'clf_chg_line_left_raspberry', 'clf_agg_turn_left_raspberry'])"

"Near crash select before check size of data (tamaño [39, 16]):"
[array([15346., 15347., 15348., 15349., 15350., 15351., 15352., 15353.,
       15354., 15355., 15356., 15357., 15358., 15359., 15360., 15361.,
       15362., 15363., 15364., 15365., 15366., 15367., 15368., 15369.,
       15370., 15371., 15372., 15373., 15374., 15375., 15376., 15383.,
       15384., 15385., 15386., 15387., 15389., 15390., 15391.]), array([15762., 15763., 15764., 15768., 15769., 15770., 15771., 15772.,
       15773., 15774., 15775., 15776., 15777., 15778., 15779., 15780.])]

"Near crash select before check size of data (tamaño [14, 72, 27, 16, 27]):"
[array([15181., 15182., 15183., 15186., 15187., 15188., 15189., 15190.,
       15191., 15192., 15193., 15203., 15204., 15205.]), array([15421., 15422., 15423., 15424., 15425., 15426., 15427., 15428.,
       15429., 15430., 15431., 15432., 15433., 15434., 15435., 15436.,
       15437., 15438., 15439., 15440., 15441., 15442., 15443., 15444.,
       15445., 15446., 15447., 15448., 15449., 15450., 15451., 15452.,
       15453., 15454., 15455., 15456., 15457., 15458., 15459., 15460.,
       15461., 15462., 15463., 15464., 15465., 15466., 15467., 15468.,
       15469., 15470., 15471., 15472., 15473., 15474., 15475., 15476.,
       15477., 15478., 15479., 15480., 15481., 15482., 15483., 15484.,
       15485., 15486., 15487., 15488., 15489., 15490., 15491., 15492.]), array([15842., 15843., 15844., 15845., 15846., 15847., 15848., 15849.,
       15853., 15854., 15855., 15856., 15857., 15858., 15859., 15860.,
       15861., 15862., 15863., 15864., 15865., 15866., 15873., 15874.,
       15875., 15876., 15877.]), array([16534., 16535., 16536., 16538., 16539., 16540., 16541., 16542.,
       16543., 16544., 16545., 16546., 16547., 16548., 16549., 16550.]), array([18104., 18105., 18106., 18107., 18108., 18109., 18110., 18111.,
       18112., 18113., 18114., 18115., 18116., 18117., 18118., 18119.,
       18120., 18121., 18122., 18123., 18124., 18125., 18126., 18127.,
       18128., 18132., 18133.])]

"Near crash select before check size of data (tamaño [20, 7]):"
[array([15172., 15173., 15174., 15175., 15176., 15177., 15178., 15179.,
       15180., 15181., 15182., 15183., 15184., 15185., 15186., 15187.,
       15188., 15189., 15190., 15191.]), array([16342., 16343., 16344., 16345., 16346., 16367., 16368.])]

"Near crash select before check size of data (tamaño [18]):"
[array([16349., 16350., 16351., 16352., 16353., 16354., 16355., 16356.,
       16357., 16358., 16359., 16360., 16361., 16362., 16363., 16364.,
       16365., 16366.])]


"Near crash select before check size of data (tamaño [21, 15, 47]):"
[array([16345., 16346., 16347., 16348., 16349., 16350., 16351., 16352.,
       16353., 16354., 16355., 16356., 16357., 16358., 16359., 16360.,
       16361., 16362., 16363., 16364., 16365.]), array([17211., 17212., 17213., 17214., 17215., 17216., 17217., 17218.,
       17219., 17220., 17221., 17222., 17223., 17224., 17225.]), array([18019., 18020., 18021., 18022., 18023., 18024., 18025., 18026.,
       18027., 18028., 18029., 18030., 18031., 18032., 18033., 18034.,
       18035., 18036., 18037., 18038., 18039., 18040., 18041., 18042.,
       18043., 18044., 18045., 18046., 18047., 18048., 18049., 18050.,
       18051., 18052., 18053., 18054., 18055., 18056., 18057., 18058.,
       18059., 18060., 18061., 18062., 18063., 18064., 18065.])]

In [ ]:
    """
    for id in output.values():
        first_id = id.index.get_level_values('first')  # Get the first index for every sliding window
        event_ocurrence = np.split(first_id.values, np.where(np.diff(first_id.values) != stepsize)[0]+1) # Split consecutive data

        len_event_ocurrence = [len(x) for x in event_ocurrence]
        print("\n\n\n")
        print("Near crash split (tamaño {}):\n{}".format(len_event_ocurrence,event_ocurrence))

        # Join separate events that correspond to oneself
        joined_events = [event_ocurrence[0]]
        for i, d in enumerate(event_ocurrence[:-1]):
            if (event_ocurrence[i][-1] + windows_size) >= event_ocurrence[i+1][0]:
                joined_events[-1] = np.concatenate((joined_events[-1], event_ocurrence[i+1]))
            else:
                joined_events.append(event_ocurrence[i+1])

        len_joined_events = [len(x) for x in joined_events]
        print("\nNear crash join split data (tamaño {}):\n{}"
              .format(len_joined_events,joined_events))

        # Select the events that comply with at least 50 records
        select_near_crash = [x for x in joined_events if (x[-1] + windows_size)-x[0] >= register_number] # Select the near crash that satisfy the minimum records condition
        
        len_s_nc = [len(x) for x in select_near_crash]
        print("\nNear crash select before check size of data (tamaño {}):\n{}"
              .format(len_s_nc, select_near_crash))
    """

In [13]:
import pandas as pd
import numpy as np

arrays = [
    [1, 2, 3, 4, 5, 6, 7, 8],
    [5, 6, 7, 8, 9, 10, 11, 12],
]

tuples = list(zip(*arrays))

index = pd.MultiIndex.from_tuples(tuples, names=["first", "second"])
df = pd.DataFrame(np.random.randn(6, 6), index=index[:6])
df

,,0,1,2,3,4,5
first,second,,,,,,
1,5,1.754319,0.038805,0.296372,-2.389115,1.029616,0.349348
2,6,1.042180,-1.053406,-1.692715,-0.173456,-0.498795,-0.549838
3,7,0.100411,-0.203316,-0.328401,0.343012,-0.873099,1.486027
4,8,0.025793,0.333855,1.529303,-0.648195,0.562014,-2.207788
5,9,-1.124014,0.119476,-1.268884,-0.333410,0.790083,-0.550608
6,10,-1.370188,0.387451,2.857159,0.443767,-2.482349,0.132429


In [22]:
df.loc[np.array([1,3,6]), np.array([4])].index.get_level_values('second')

Int64Index([5, 7, 10], dtype='int64', name='second')

In [ ]:
"Near crash select before check size of data (tamaño [25, 111, 16, 27, 27, 16, 15, 47, 27]):"
[array([15172., 15173., 15174., 15175., 15176., 15177., 15178., 15179.,
       15180., 15181., 15182., 15183., 15184., 15185., 15186., 15187.,
       15188., 15189., 15190., 15191., 15192., 15193., 15203., 15204.,
       15205.]), array([15346., 15347., 15348., 15349., 15350., 15351., 15352., 15353.,
       15354., 15355., 15356., 15357., 15358., 15359., 15360., 15361.,
       15362., 15363., 15364., 15365., 15366., 15367., 15368., 15369.,
       15370., 15371., 15372., 15373., 15374., 15375., 15376., 15383.,
       15384., 15385., 15386., 15387., 15389., 15390., 15391., 15421.,
       15422., 15423., 15424., 15425., 15426., 15427., 15428., 15429.,
       15430., 15431., 15432., 15433., 15434., 15435., 15436., 15437.,
       15438., 15439., 15440., 15441., 15442., 15443., 15444., 15445.,
       15446., 15447., 15448., 15449., 15450., 15451., 15452., 15453.,
       15454., 15455., 15456., 15457., 15458., 15459., 15460., 15461.,
       15462., 15463., 15464., 15465., 15466., 15467., 15468., 15469.,
       15470., 15471., 15472., 15473., 15474., 15475., 15476., 15477.,
       15478., 15479., 15480., 15481., 15482., 15483., 15484., 15485.,
       15486., 15487., 15488., 15489., 15490., 15491., 15492.]), array([15762., 15763., 15764., 15768., 15769., 15770., 15771., 15772.,
       15773., 15774., 15775., 15776., 15777., 15778., 15779., 15780.]), array([15842., 15843., 15844., 15845., 15846., 15847., 15848., 15849.,
       15853., 15854., 15855., 15856., 15857., 15858., 15859., 15860.,
       15861., 15862., 15863., 15864., 15865., 15866., 15873., 15874.,
       15875., 15876., 15877.]), array([16342., 16343., 16344., 16345., 16346., 16347., 16348., 16349.,
       16350., 16351., 16352., 16353., 16354., 16355., 16356., 16357.,
       16358., 16359., 16360., 16361., 16362., 16363., 16364., 16365.,
       16366., 16367., 16368.]), array([16534., 16535., 16536., 16538., 16539., 16540., 16541., 16542.,
       16543., 16544., 16545., 16546., 16547., 16548., 16549., 16550.]), array([17211., 17212., 17213., 17214., 17215., 17216., 17217., 17218.,
       17219., 17220., 17221., 17222., 17223., 17224., 17225.]), array([18019., 18020., 18021., 18022., 18023., 18024., 18025., 18026.,
       18027., 18028., 18029., 18030., 18031., 18032., 18033., 18034.,
       18035., 18036., 18037., 18038., 18039., 18040., 18041., 18042.,
       18043., 18044., 18045., 18046., 18047., 18048., 18049., 18050.,
       18051., 18052., 18053., 18054., 18055., 18056., 18057., 18058.,
       18059., 18060., 18061., 18062., 18063., 18064., 18065.]), array([18104., 18105., 18106., 18107., 18108., 18109., 18110., 18111.,
       18112., 18113., 18114., 18115., 18116., 18117., 18118., 18119.,
       18120., 18121., 18122., 18123., 18124., 18125., 18126., 18127.,
       18128., 18132., 18133.])]